In [1]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import decomposition
from matplotlib import pyplot as plt
from sklearn import neighbors

from double_CV_builder import DCV

In [2]:
# imports the data
data = pd.read_csv("../raw_data/train_call.txt", index_col=0, delimiter="\t").transpose()
topInfo = data.iloc[:3]
data = data.iloc[4:].dropna(axis=1)
classes = pd.read_csv("../raw_data/train_clinical.txt", delimiter="\t", index_col=0)
data = classes.join(data).set_index("Subgroup").dropna()
# Add 1 to data because of chi-square feature selection


DCV.outer_repeats = 5
DCV.inner_repeats = 10
DCV.num_features = 150

model = linear_model.LogisticRegression(penalty="elasticnet", max_iter=10000, multi_class='ovr', solver='saga')
modeller = DCV(model)
# prints all the params you can change :)
print(model.get_params().keys())
# select which params you want to test with the inner loop
# space is the paramater space
modeller.hyperParams['C'] = [100, 10, 1.0, 0.1, 0.01]
modeller.hyperParams['l1_ratio'] = [0.1, 0.3, 0.5, 0.7, 0.9]



dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])


In [3]:
# this does everything for you :)
# first is the data used, seconds comes the classifications and than with loop the amount of loops you want to do

modeller.train_fit(data=data, classes=data.index, loop=1)
# some test code

print(accuracy_score(DCV.__class_cleaner__(None, data.index),modeller.predict(data)))

(99, 1352)
accuracy: 0.55; precision:0.55; model:LogisticRegression(l1_ratio=0.9, max_iter=10000, multi_class='ovr',
                   penalty='elasticnet', solver='saga')
accuracy: 0.55; precision:0.55; model:LogisticRegression(C=10, l1_ratio=0.9, max_iter=10000, multi_class='ovr',
                   penalty='elasticnet', solver='saga')
accuracy: 0.55; precision:0.55; model:LogisticRegression(l1_ratio=0.5, max_iter=10000, multi_class='ovr',
                   penalty='elasticnet', solver='saga')
accuracy: 0.6; precision:0.6; model:LogisticRegression(C=10, l1_ratio=0.1, max_iter=10000, multi_class='ovr',
                   penalty='elasticnet', solver='saga')
accuracy: 0.3; precision:0.3; model:LogisticRegression(l1_ratio=0.9, max_iter=10000, multi_class='ovr',
                   penalty='elasticnet', solver='saga')
(99, 1352)
0.98989898989899


In [4]:
print(topInfo.transpose().iloc[:-1].loc[modeller.get_best_features()[0]])

                  Start          End  Nclone
Chromosome                                  
1            30716764.0   30984527.0    19.0
1            31136968.0   31957740.0    91.0
1            72489681.0   72620747.0     5.0
1           149369522.0  149394958.0     5.0
2            61025070.0   61061211.0     5.0
...                 ...          ...     ...
8            41575958.0   42118548.0    55.0
8            42123757.0   42175380.0     9.0
8            48952907.0   49214104.0    27.0
8            49291873.0   50106229.0    46.0
8            50133866.0   50469104.0    16.0

[150 rows x 3 columns]


In [5]:
modeller.all_accuracies

0
C      l1_ratio        
100.00 0.1       0.6750
       0.3       0.6625
       0.5       0.6625
       0.7       0.6625
       0.9       0.6625
...                 ...
0.01   0.1       0.3750
       0.3       0.3500
       0.5       0.3625
       0.7       0.3625
       0.9       0.3375

[125 rows x 1 columns]

In [6]:
modeller.all_accuracies.reset_index().pivot_table(index="l1_ratio",columns="C", aggfunc="mean")["Value"]

KeyError: 'Value'

In [ ]:
cm = confusion_matrix(data.index, modeller.predict(data))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=modeller.get_best_model().classes_)
disp.plot()

In [ ]:
import scipy.stats as stats

stats.wilcoxon(modeller.predict(data), data.index)

In [ ]:
print(modeller.get_best_features())


In [ ]:
modeller.all_accuracies
modeller.all_precision
modeller.all_recall

#todo
box plots accuracy, precision, recall
best model. and params features

save all the juicy info (params, and features)

In [ ]:
print(model.__callable__("predict_proba")==True)